In [134]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [135]:
df = pd.read_csv('../Video_games_esrb_rating.csv')

In [136]:
df.head()

,title,console,alcohol_reference,animated_blood,blood,blood_and_gore,cartoon_violence,crude_humor,drug_reference,fantasy_violence,...,sexual_content,sexual_themes,simulated_gambling,strong_janguage,strong_sexual_content,suggestive_themes,use_of_alcohol,use_of_drugs_and_alcohol,violence,esrb_rating
0,Monster Jam Steel Titans 2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
1,Subnautica: Below Zero,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ET
2,NIER REPLICANT VER.1.22474487139…,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,M
3,Jamestown+,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,ET
4,Neptunia Virtual Stars,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,T


In [137]:
col_val = df.drop(['title', 'esrb_rating'], axis=1)

#Correcting typo in the original dataframe
df = df.rename(columns={"strong_janguage": "strong_language"})

In [138]:
# Checking if the int64 variables have only 0's and 1's
unique_val = np.unique(col_val)
unique_val

array([0, 1], dtype=int64)

In [139]:
del col_val, unique_val

In [140]:
ratings = df['esrb_rating'].unique()
ratings

array(['E', 'ET', 'M', 'T'], dtype=object)

In [141]:
# Checking if we have repeated video game titles
df['title'].is_unique

True

In [142]:
df['esrb_rating'] = df['esrb_rating'].replace(
    'E', 0).replace('ET', 1).replace('T', 2).replace('M', 3)

In [143]:
y = df['esrb_rating']
X = df.drop(['esrb_rating'], axis=1)

In [144]:
SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=SEED)

In [145]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

ValueError: could not convert string to float: 'GALAK-Z: The Void'

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 96   2   1   0]
 [  4  49   8   0]
 [  2  18 117   9]
 [  0   0  14  59]]
              precision    recall  f1-score   support

           0       0.94      0.97      0.96        99
           1       0.71      0.80      0.75        61
           2       0.84      0.80      0.82       146
           3       0.87      0.81      0.84        73

    accuracy                           0.85       379
   macro avg       0.84      0.85      0.84       379
weighted avg       0.85      0.85      0.85       379

